# load and explore data

In [ ]:
from data import store,raw_data

In [ ]:
store

In [ ]:
print 'common_feat :',raw_data.common_feat
print 'label_name  :',raw_data.label_name

In [ ]:
raw_data.store is store

# design a extractor 
- uncomment to save it into file

In [ ]:
#%%writefile competition/extractors/sf_partial_2.py
from competition.extractors.base import BaseExtractor
import pandas as pd
import os
class SFPartial2(BaseExtractor):
    def get_train(self,raw_data):
        self.init(raw_data)
        X,y = self._get(self.raw_data.store['train'])
        idx = X['clickTime']<190000
        X=X[idx]
        y=y[idx]
        X = self.merge_tables(X)
        return X,y

    def get_test(self,raw_data):
        self.init(raw_data)
        X,y = self._get(self.raw_data.store['test'])
        X = self.merge_tables(X)
        return X,y

    def _get(self,dataframe):
        X = dataframe[self.raw_data.common_feat]
        y = dataframe[self.raw_data.label_name]
        return X,y

    def init(self, raw_data):
        if hasattr(self,'bInit'):
            return
        self.raw_data = raw_data
        store = raw_data.store
        f_user = store['feat-user-default']
        f_ad = store['feat-ad-default']
        f_pos = store['position']
        def merge_tables(X):
            # user
            X = pd.merge(X,f_user,how='left',left_on='userID',right_on='userID')
            del X['userID']
            # ad
            X = pd.merge(X,f_ad,how='left',left_on='creativeID',right_on='creativeID')
            del X['creativeID']
            # pos
            X = pd.merge(X,f_pos,how='left',left_on='positionID',right_on='positionID')
            del X['positionID']

            return X
        self.merge_tables = merge_tables
        self.bInit=True

# must provide extractor and extractor_name
extractor = SFPartial2()
extractor_name = SFPartial2.__name__

In [ ]:
trainX,trainY = extractor.get_train(raw_data)

In [ ]:
print trainX.shape
print trainX.columns
trainX.head()

In [ ]:
testX,testY  = extractor.get_train(raw_data)

In [ ]:
print testX.shape
print testX.columns
testX.head()

# run the extractor
- save it into file
- modify `configCustom.py`
- run `run_1_extract_features.py`

In [ ]:
%%writefile competition/extractors/sf_partial_2.py
from competition.extractors.base import BaseExtractor
import pandas as pd
import os
class SFPartial2(BaseExtractor):
    def get_train(self,raw_data):
        self.init(raw_data)
        X,y = self._get(self.raw_data.store['train'])
        idx = X['clickTime']<190000
        X=X[idx]
        y=y[idx]
        X = self.merge_tables(X)
        return X,y

    def get_test(self,raw_data):
        self.init(raw_data)
        X,y = self._get(self.raw_data.store['test'])
        X = self.merge_tables(X)
        return X,y

    def _get(self,dataframe):
        X = dataframe[self.raw_data.common_feat]
        y = dataframe[self.raw_data.label_name]
        return X,y

    def init(self, raw_data):
        if hasattr(self,'bInit'):
            return
        self.raw_data = raw_data
        store = raw_data.store
        f_user = store['feat-user-default']
        f_ad = store['feat-ad-default']
        f_pos = store['position']
        def merge_tables(X):
            # user
            X = pd.merge(X,f_user,how='left',left_on='userID',right_on='userID')
            del X['userID']
            # ad
            X = pd.merge(X,f_ad,how='left',left_on='creativeID',right_on='creativeID')
            del X['creativeID']
            # pos
            X = pd.merge(X,f_pos,how='left',left_on='positionID',right_on='positionID')
            del X['positionID']

            return X
        self.merge_tables = merge_tables
        self.bInit=True

# must provide extractor and extractor_name
extractor = SFPartial2()
extractor_name = SFPartial2.__name__

In [ ]:
%%writefile configCustom.py
dataDir = './_data/'
extractDir = './_extracted/'
resultDir = './_result/'

dataFile = 'store.db'
infoFile = 'info.pkl'

# extractor
from competition.extractors.sf_partial_2 import extractor,extractor_name

# model
from competition.models.impute_xgb import estimator,estimator_name

tuned_parameters = [{
    'impute__copy':[False],
    'xgb__colsample_bytree': [0.5],
    'xgb__gamma': [0],
    'xgb__learning_rate': [0.1],
    'xgb__max_depth': [3],
    'xgb__min_child_weight': [1],
    'xgb__objective': ['binary:logistic'],
    'xgb__subsample': [1]
},]

# other configs
bShuffle = True
bProb = True


In [ ]:
%run run_1_extract_features.py